In [1]:
import numpy as np
import pandas as pd
import requests
import bs4
import re
from time import sleep
from spellchecker import SpellChecker

In [16]:
result = requests.get("https://en.wikipedia.org/wiki/List_of_jazz_organists")

In [17]:
soup = bs4.BeautifulSoup(result.text, "html.parser")

In [18]:
uOrganistsLinks = dict()

# For each letter category on the page (which are marked by div class: div-col)
for div_col in soup.find_all('div', {'class' : 'div-col'}):
    # For each tag containing the artist's name and hyperlink (marked by the <a> tag)
    for artist_tag in div_col.find_all('a'):
        # artist_tag.text = artist's name; artist_tag.attrs["href"] = wiki link extension for that artist
        #print(f'{artist_tag.text}, {artist_tag.attrs["href"]}')
        uOrganistsLinks.update( {artist_tag.text : artist_tag.attrs["href"] } )
            


In [19]:
# There aren't years listed on this list either (similar to the guitarists). More manual data collection! Yay!

In [20]:
# Creating a custom dictionary with pyspellchecker that contains most, if not all, instruments used to play jazz
spell = SpellChecker(language = None, case_sensitive = True)
a = spell.word_frequency.load_words(['Piano', 'Vocals', 'Keyboards', 'Double bass', 'Saxophone', 'Clarinet', 'Trumpet', 
                                    'Trombone', 'Drums', 'Guitar', 'Flute', 'Vibraphone', 'Banjo', 'Violin', 'Viola', 
                                    'Cello', 'Synthesizer'])

In [21]:
# Creating DataFrame to store the information on the pianists
pDF = pd.DataFrame(columns = ['artist', 'instrument', 'birthdate', 'deathdate', 'birth_year', 'death_year'])

# Main script that will take the wikis in the fullPianistsList dictionary and scrape the data in the biographical
# table for each artist. I have done much research into the legality and ethicality of this operation, and I've 
# determined that it's okay to run a script like this as long as I have at least a one-second delay inbetween 
# each iteration of the below for loop, so as to not create too much traffic. 

# For testing the DataFrame:
runs = 0

for organist, link in uOrganistsLinks.items():
    
    # For testing purposes
    #if runs < 30:
        #print(organist, link)
    
        bday = ''
        dday = ''
        
        bYear = ''
        dYear = ''
        
        lSpan = 0
        
        # instrumentList is important for a later operation, although it can most likely be simplified
        instrumentList = []
    
        # Create the link by appending the wikipedia link for the artist onto the end of the general wikipedia link
        wikiLink = 'https://en.wikipedia.org' + link
    
        # Following the basic requests and bs4 process to read in a webpage
        pInfo = requests.get(wikiLink)
        pSoup = bs4.BeautifulSoup(pInfo.text, "html.parser")
    
        # Finding the biographical information table, which is always in a table with the class = infobox vcard plainlist
        # However, some pages don't have the table, which is why there is a conditional first. 
        if pSoup.find('table', {'class' : 'infobox vcard plainlist'}):
            
    
            # aTable is the tag in which the script will find all of the necessary information
            aTable = pSoup.find('table', {'class' : 'infobox vcard plainlist'})
    
            # All of the necessary information is within the <tr> row element 
            for table in aTable.find_all('tr'):
    
                # First, finding the primary instrument of the artist (which is all that this project will
                # be concerned with for the moment, sorry multi-instrumentalists). The instrument data is listed in
                # the <td> tag with the class = note. However, because the script must account for the fact that
                # there are multi-instrumentalists, the script only returns the first instrument that is found. 
                # In lieu of a more efficient way to do this, there is an index that will only allow the instrumentList
                # to append the first instrument it finds. This should probably be fixed too. 
                if table.find('td', {'class' : 'note'}):
                    index = 0
                    for string in table.find('td', {'class' : 'note'}).stripped_strings:
                        if index == 0:
                            string = string.split(',')[0]
                            string = spell.correction(string)
                            instrumentList.append(string.split(',')[0])
                            index += 1
        
                # Finding the birthday of the artist. If there is one, it is always contained within the span class = 
                # bday tag, and the date can be accessed with the .string bs4 method
                # THERE ARE ISSUES HERE BECAUSE OF WIKIPEDIA SHODDY FORMATTING - FIGURE THIS OUT
                if table.find('span', {'class' : 'bday'}):
                    #print(table.find('span', {'class' : 'bday'}).string)
                    bday = table.find('span', {'class' : 'bday'}).string
                    bYear = bday[0:4]
        
                # Finding the deathdate of the artist. If there is one, it is usually contained within the span style = 
                # display:none tag. However, there are multiple pieces of information that can be contained within a 
                # style = diplay:none tag. Fortunately, the death date is the only piece of information that is 
                # accessible via the .string method, so the script checks for the possibility of the .string method
                # and then removes the parentheses which always surround it. 
                if table.find('span', {'style' : 'display:none'}):
                    for row in table.find_all('span', {'style' : 'display:none'}):
                        if row.string:
                            #print(row.string[1:-1])
                            dday = row.string[1:-1]
                            dYear = dday[0:4]
                    
     
            if not bday:
                bday = 'NaN'
                bYear = bday
                
            if not dday:
                dday = 'NaN'
                dYear = 'NaN'
            
            if not instrumentList:
                instrumentList.append('Organ')
        
            #print('\n')
        
            # Time delay on for loop
            sleep(1.0)
            
            
    
        else:
            #print('\n')
            bday = 'NaN'
            bYear = bday
            dday = 'NaN'
            dYear = dday
                
            bday = 'NaN'
            bYear = bday
            if bday == 'NaN':
                dday = 'NaN'
                dYear = 'NaN'
                    
            if not instrumentList:
                instrumentList.append('Organ')
                
                
        # Appending the values to the DataFrame        
        pDF = pDF.append( {'artist' : organist, 'instrument' : instrumentList[0], 'birthdate' : str(bday),
                           'deathdate' : str(dday), 'birth_year' : bYear, 'death_year' : dYear}, ignore_index = True)
    
        runs += 1
        #print(runs)



In [22]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
# To make it easier to loop through DataFrame? 
# pDF.set_index('Artist', inplace = True)

# Deleting Birth Year and Death Year columns for now because I don't think they're that necessary given the solution
# I've decided to pursue. However, I will leave this like so for now because I don't want to commit to this solution 
# yet. 
pDF.drop(columns = ['birth_year', 'death_year'], inplace = True)

#print(pDF)

In [23]:
# Adding 'alive' column to the DataFrame using np.select() 

conditions = [ (pDF['birthdate'] == 'NaN') & (pDF['deathdate'] == 'NaN'), (pDF['deathdate'] == 'NaN'), 
             (pDF['deathdate'] != 'NaN') ]

values = ['NaN', True, False]

pDF['alive'] = np.select(conditions, values)
#print(pDF)

In [24]:
# Forcing all of the dates into datetime objects 

pDF['bday_dt'] = pd.to_datetime(pDF['birthdate'], errors = 'coerce')
pDF['dday_dt'] = pd.to_datetime(pDF['deathdate'], errors = 'coerce')

# Account for the age of people who are alive today as well
conditions = [ (pDF['alive'] == 'True'), (pDF['alive'] == 'False'), (pDF['alive'] == 'NaN') ]

values = [ (pd.to_datetime('today') - pDF['bday_dt']).astype('timedelta64[Y]').astype(float),
         (pDF['dday_dt'] - pDF['bday_dt']).astype('timedelta64[Y]').astype(float), 'NaN' ]

pDF['age_yrs'] = np.select(conditions, values)

#display(pDF)

In [15]:
# Outputs the .csv file that was then cleaned in Excel and put back into the jazz-pianist-files folder as 
# jazz_pianists_cleaned1.csv
pDF.to_csv('jazz_organists1.csv', index = False)